<a href="https://colab.research.google.com/github/NeuralClassifier/ReusabilityRate_ML/blob/main/CMPUT_663_ReuseRate_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 

In [2]:
!git clone https://github.com/NeuralClassifier/ReusabilityRate_ML

Cloning into 'ReusabilityRate_ML'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [16]:
df = pd.read_excel('/content/ReusabilityRate_ML/DataSets/packages.xlsx')
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [18]:
df

,Project,LongName,Name,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL,ReuseRate
0,ch.qos.logback/logback-classic,<root_package>,<root_package>,0.000000,0.000000,0.342362,0,4146,0,219,0,0,14288,0,7964,403,217,853,3498,624,0,0
1,ch.qos.logback/logback-classic,ch,ch,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0,0
2,ch.qos.logback/logback-classic,unnamed package,unnamed package,0.000000,0.000000,0.000000,0,0,0,0,32,38,38,0,32,0,0,0,0,0,0,0
3,ch.qos.logback/logback-classic,ch.qos,qos,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0,0
4,ch.qos.logback/logback-classic,ch.qos.logback,logback,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,xerces/xercesImpl,org.apache.xerces.impl.xs.identity,identity,0.984127,0.498253,0.498253,713,713,62,62,718,1603,1603,22,718,39,22,66,394,53,10,0
2189,xerces/xercesImpl,org.apache.xerces.impl.xs.models,models,0.701493,0.476530,0.476530,1137,1137,47,47,1249,2719,2719,17,1249,52,17,91,732,59,10,0
2190,xerces/xercesImpl,org.apache.xerces.impl.xs.opti,opti,0.540636,0.482071,0.482071,1909,1909,153,153,2051,4621,4621,94,2051,96,94,312,851,280,15,3
2191,xerces/xercesImpl,org.apache.xerces.impl.xs.traversers,traversers,0.700000,0.225403,0.225403,2543,2543,14,14,8739,12621,12621,34,8739,332,34,290,4838,57,29,0


In [20]:
df.iloc[:,3:-1]

,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL
0,0.000000,0.000000,0.342362,0,4146,0,219,0,0,14288,0,7964,403,217,853,3498,624,0
1,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
2,0.000000,0.000000,0.000000,0,0,0,0,32,38,38,0,32,0,0,0,0,0,0
3,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
4,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,0.984127,0.498253,0.498253,713,713,62,62,718,1603,1603,22,718,39,22,66,394,53,10
2189,0.701493,0.476530,0.476530,1137,1137,47,47,1249,2719,2719,17,1249,52,17,91,732,59,10
2190,0.540636,0.482071,0.482071,1909,1909,153,153,2051,4621,4621,94,2051,96,94,312,851,280,15
2191,0.700000,0.225403,0.225403,2543,2543,14,14,8739,12621,12621,34,8739,332,34,290,4838,57,29


In [26]:
class ReuseTradML:
  def __init__(self,data,dataset_name):
    self.ml_algo_list = ['MLPRegressor',
                         'LogisticRegression',
                         'BayesianRidge',
                         'SGDRegressor',
                         'XGBRegressor',
                         'DecisionTreeRegressor']

    self.metrics = ['RMSE',
                    'MAE']

    if dataset_name == 'packages':
      target_scaler = MinMaxScaler()
      self.X = np.array(data.iloc[:,3:-1])
      target_scaler.fit(self.X)
      self.X = target_scaler.transform(self.X)

      self.Y = data.iloc[:,-1]
      self.Y = self.Y.values.reshape(-1, 1)
      target_scaler = MinMaxScaler()
      target_scaler.fit(self.Y)
      self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]

    elif dataset_name == 'classes':
      target_scaler = MinMaxScaler()
      self.X = np.array(data.iloc[:,3:-1])
      target_scaler.fit(self.X)
      self.X = target_scaler.transform(self.X)

      self.Y = data.iloc[:,-1]
      self.Y = self.Y.values.reshape(-1, 1)
      target_scaler = MinMaxScaler()
      target_scaler.fit(self.Y)
      self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]
      
    elif dataset_name == 'projects':
      self.X = np.array(data.iloc[:,4:])
      self.other = data.iloc[:,:4]

    else:
      print('Error, Please try again!')
      self.X=None
      self.Y=None
      self.other=None

  def apply_ml(self):
    while cnt < self.X.shape[1]:

    return self.X, self.Y, self.other

In [27]:
rtm=ReuseTradML(df,dataset_name='packages')
x,y,oth=rtm.apply_ml()
x

array([[0.        , 0.        , 0.37227707, ..., 0.02903338, 0.01859412,
        0.        ],
       [0.        , 0.        , 0.37326332, ..., 0.02903338, 0.01859412,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.540636  , 0.49992533, 0.52419363, ..., 0.0070633 , 0.00834351,
        0.01679731],
       [0.7       , 0.23375119, 0.24509837, ..., 0.04015538, 0.0016985 ,
        0.0324748 ],
       [0.458716  , 0.41749958, 0.43776661, ..., 0.0034362 , 0.00378438,
        0.02015677]])

In [29]:
x.shape

(2193, 18)